<a href="https://colab.research.google.com/github/dowrave/Tensorflow_Basic/blob/main/220513_tf_variables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## tf.Variable
- 변수는 `tf.Variable` 클래스로 생성 및 추적된다.
  - 변수 = ops를 실행하여 <b>값을 변경할 수 있는</b> 텐서
  - `tf.keras`는 `tf.Variable`을 사용하여 모델 매개변수를 저장한다.

In [2]:
import tensorflow as tf

In [4]:
my_tensor = tf.constant([[1.0, 2.0], [3.0, 4.0]])
my_variable = tf.Variable(my_tensor) # constant -> variable

bool_variable = tf.Variable([False, False, False, True])
complex_variable = tf.Variable([5 + 4j, 6 + 1j])

- 변수는 텐서처럼 보이고 작동하며 실제로 `tf.Tensor`에서 지원되는 데이터 구조이다. `dtype`, `shape`를 가지며 `numpy`로 내보낼 수 있음

In [6]:
my_variable.shape, my_variable.dtype, my_variable.numpy

(TensorShape([2, 2]),
 tf.float32,
 <bound method BaseResourceVariable.numpy of <tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
 array([[1., 2.],
        [3., 4.]], dtype=float32)>>)

In [8]:
# 텐서 재할당 - 기존 텐서의 메모리가 재사용된다
a = tf.Variable([2.0, 3.0])

a.assign([1, 2]) # 값을 바꾸는 건 가능하지만

try:
  a.assign([1., 2., 3.]) # 구조를 바꾸는 건 불가능하다
except Exception as e:
  print(f"{type(e).__name__}: {e}")

ValueError: Cannot assign value to variable ' Variable:0': Shape mismatch.The variable shape (2,), and the assigned value shape (3,) are incompatible.


## 텐서 복제
- 텐서와 같은 변수를 사용하면, 지원 텐서가 복제된다. (같은 메모리를 쓰지 않는다)

In [9]:
a = tf.Variable([2.0, 3.0])
b = tf.Variable(a) # tf.Variable로 할당하면 복제되지만 (= 다른 메모리 주소를 사용)
c = a # 이거는 같은 메모리 주소를 참조하게 된다.

a.assign([5, 6])

print(a.numpy())
print(b.numpy())
print(c.numpy())

print(a.assign_add([2, 3]).numpy())
print(a.assign_sub([7, 9]).numpy())

[5. 6.]
[2. 3.]
[5. 6.]
[7. 9.]
[0. 0.]


## 수명 주기
- `tf.Variable`은 다른 파이썬 객체와 똑같이, 변수에 대한 참조가 없으면 할당이 자동으로 해제된다.
- 변수 추적 & 디버그 시 도움이 되는 변수의 이름을 지정할 수 있다. 두 변수에 같은 이름을 지정할 수도 있다.
  - 변수의 이름은 보통 자동 지정되기 때문에 `name=`식으로 설정하는 경우는 잘 없음

In [11]:
a = tf.Variable(my_tensor, name = "mark")
b = tf.Variable(my_tensor + 1, name = "mark")  # my_tensor이면 True임

print(a==b) # 두 변수의 이름이 동일하지만, 다릅니다


tf.Tensor(
[[ True  True]
 [ True  True]], shape=(2, 2), dtype=bool)


- 한편 `trainable`을 관리할 필요는 있음. 그래디언트가 필요 없는 변수들이 있기 때문(ex : 훈련 카운터)

In [12]:
step_counter = tf.Variable(1, trainable = True)

## 변수와 텐서 배치하기
- Tensorflow는 dtype이 호환되는 가장 빠른 기기에 텐서, 변수를 배치하려고 시도함
  - 즉 대부분의 경우 GPU에 배치되려고 함
  - 재정의할 수 있음
- 수동 배치도 가능하지만, 분배 전략을 사용하면 계산 최적화에 더 편리하고 확장 가능한 방법이 될 수 있음

In [14]:
with tf.device('CPU:0'):

  a = tf.Variable([[1., 2., 3.], [4.,  5. , 6.,]])
  b = tf.constant([[1., 2.], [3., 4.], [5. ,6.]])
  c = tf.matmul(a, b)

print(c)

tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [15]:
# Variable 할당은 CPU에서, 연산은 GPU에서 실행
with tf.device("CPU:0"):
  a = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
  b = tf.Variable([[1., 2., 3.]])

with tf.device("GPU:0"):
  k = a * b

print(k)

tf.Tensor(
[[ 1.  4.  9.]
 [ 4. 10. 18.]], shape=(2, 3), dtype=float32)


- `tf.config.set_soft_device_placement`가 기본적으로 켜져 있어서, GPU가 없는 기기에서 이 코드를 실행한다면 아래의 GPU 연산은 CPU에서 실행된다.